## MVP Engenharia de Dados

**Nome:** Bianca Carvalho Lima

**Matrícula:** 4052025000297

- **Dataset Original (de 2007 a 2020):** https://www.kaggle.com/datasets/equeiroz/acidentes-rodovias-federais-brasil-jan07-a-jul19
- **Dataset Original (de 2023 a 2025):** https://www.kaggle.com/datasets/jairsouza/acidentes-rodovias-federais

**Repo GitHub:** https://github.com/biaacarvalhoo27/PUCRIO_MVP_eng_dados

## Coleta dos dados
Análise de dados sobre os acidentes de trânsito nas rodovias federais brasileiras durante o período de tempo de Janeiro de 2007 até Janeiro de 2025. Os datasets foram disponibilizados pelo Plano de Dados Abertos da PRF através do site: https://www.gov.br/prf/pt-br/acesso-a-informacao/dados-abertos/dados-abertos-da-prf na categoria de Agrupados por ocorrência.

## Instalação de pacote Kaggle

In [0]:
%pip install kagglehub
import kagglehub

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Import dos datasets

In [0]:
acidentes_rodovias_federais_2007_2020_path = kagglehub.dataset_download('equeiroz/acidentes-rodovias-federais-brasil-jan07-a-jul19')
acidentes_rodovias_federais_2021_20xx_path = kagglehub.dataset_download('jairsouza/acidentes-rodovias-federais')

print('Dataset importado com sucesso')

Dataset importado com sucesso


## Imports

In [0]:
import numpy as np 
import pandas as pd 
from pyspark.sql import SparkSession

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

##Listagem datasets csv

In [0]:
# Diretórios
caminho_base = acidentes_rodovias_federais_2007_2020_path
caminho_base_2 = acidentes_rodovias_federais_2021_20xx_path

# Arquivos originais (2007–2018, 2020)
arquivos = [
    'datatran2007.csv',
    'datatran2008.csv',
    'datatran2009.csv',
    'datatran2010.csv',
    'datatran2011.csv',
    'datatran2012.csv',
    'datatran2013.csv',
    'datatran2014.csv',
    'datatran2015.csv',
    #'datatran2016.csv', Não Usado
    'datatran2017.csv',
    'datatran2018.csv',
    'datatran2020.csv'   
]

# Novos arquivos (2023, 2024, 2025)
arquivos_2 = [
    'datatran2023.csv',
    'datatran2024.csv',
    'datatran2025.csv'
]

dataframes = []

## Processamento dos dados - Processo de coleta

In [0]:
# Processar arquivos do primeiro diretório
for arquivo in arquivos:
    caminho_completo = os.path.join(caminho_base, arquivo)
    
    try:
        df_ano = pd.read_csv(caminho_completo, delimiter=';', encoding='latin1', on_bad_lines='skip')
    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")
        continue

    print(f"{arquivo}: {len(df_ano)} linhas carregadas")

    # Extrair ano do nome do arquivo
    ano = int(arquivo.replace('datatran', '').replace('.csv', ''))

    # Converter data com base no ano
    df_ano['data'] = pd.NaT
    if ano <= 2011:
        df_ano['data'] = pd.to_datetime(df_ano['data_inversa'], format='%d/%m/%Y', errors='coerce')
    else:
        df_ano['data'] = pd.to_datetime(df_ano['data_inversa'], format='%Y-%m-%d', errors='coerce')

    falhas = df_ano['data'].isna().sum()
    if falhas > 0:
        print(f"   {falhas} datas inválidas em {arquivo} (ano {ano})")

    dataframes.append(df_ano)

# Processar arquivos do segundo diretório (2023, 2024, 2025)
for arquivo in arquivos_2:
    caminho_completo = os.path.join(caminho_base_2, arquivo)
    
    try:
        df_ano = pd.read_csv(caminho_completo, delimiter=';', encoding='latin1', on_bad_lines='skip')
    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")
        continue

    print(f"{arquivo}: {len(df_ano)} linhas carregadas")

    # Extrair ano
    ano = int(arquivo.replace('datatran', '').replace('.csv', ''))

    # Todos esses são anos recentes → formato YYYY-MM-DD
    df_ano['data'] = pd.to_datetime(df_ano['data_inversa'], format='%Y-%m-%d', errors='coerce')

    falhas = df_ano['data'].isna().sum()
    if falhas > 0:
        print(f"  {falhas} datas inválidas em {arquivo} (ano {ano})")

    dataframes.append(df_ano)

# === 3. Concatenar todos os DataFrames ===
if dataframes:
    df = pd.concat(
        dataframes,
        ignore_index=True
    )
    
# === 4. Resultado final ===
    print(f"\n Dataset consolidado: {len(df)} registros")
    print(f"   Período: {df['data'].min()} a {df['data'].max()}")
else:
    print(" Nenhum arquivo foi carregado. Verifique os caminhos e arquivos.")

/home/spark-cc089e4c-ae5b-43d6-8ffc-3a/.ipykernel/2550/command-6078046201924353-2156171893:6: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(caminho_completo, delimiter=';', encoding='latin1', on_bad_lines='skip')


datatran2007.csv: 127675 linhas carregadas


/home/spark-cc089e4c-ae5b-43d6-8ffc-3a/.ipykernel/2550/command-6078046201924353-2156171893:6: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(caminho_completo, delimiter=';', encoding='latin1', on_bad_lines='skip')


datatran2008.csv: 141043 linhas carregadas
datatran2009.csv: 158646 linhas carregadas
datatran2010.csv: 183469 linhas carregadas
datatran2011.csv: 192326 linhas carregadas


/home/spark-cc089e4c-ae5b-43d6-8ffc-3a/.ipykernel/2550/command-6078046201924353-2156171893:6: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ano = pd.read_csv(caminho_completo, delimiter=';', encoding='latin1', on_bad_lines='skip')


datatran2012.csv: 184568 linhas carregadas
datatran2013.csv: 186748 linhas carregadas
datatran2014.csv: 169201 linhas carregadas
datatran2015.csv: 122161 linhas carregadas
datatran2017.csv: 89518 linhas carregadas
datatran2018.csv: 69206 linhas carregadas
datatran2020.csv: 63530 linhas carregadas
datatran2023.csv: 67766 linhas carregadas
datatran2024.csv: 73156 linhas carregadas
datatran2025.csv: 5478 linhas carregadas

 Dataset consolidado: 1834491 registros
   Período: 2007-01-01 00:00:00 a 2025-01-31 00:00:00


## Configurar diretório schema _raw_

In [0]:
catalog_name = "lakehouse"
schema_name = "raw"
volume_name = "volumes"
directory_name = "datatran"

# Caminho completo do Volume
volume_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/{directory_name}"

# Criar o diretório se não existir
try:
    # Verificar se o diretório existe
    dbutils.fs.ls(volume_path)
    print(f"Diretório já existe: {volume_path}")
except:
    # Criar diretório
    dbutils.fs.mkdirs(volume_path)
    print(f"Diretório criado: {volume_path}")

# Mostrar estrutura
print("\nEstrutura do Volume:")
display(dbutils.fs.ls(f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/"))

Diretório já existe: /Volumes/lakehouse/raw/volumes/datatran

Estrutura do Volume:


path,name,size,modificationTime
dbfs:/Volumes/lakehouse/raw/volumes/datatran/,datatran/,0,1766201236000


## Camada _raw_ - dados brutos

In [0]:
# Iniciar sessão Spark
spark = SparkSession.builder.appName("PRF_Data_Load").getOrCreate()

## Save dos datasets individuais separados por pastas com o ano correspondente

In [0]:
# Função para limpar dados antes da conversão
def clean_pandas_dataframe(df_ano):
    """
    Limpa o DataFrame pandas antes de converter para Spark
    """
    # Colunas que precisam de tratamento especial
    if 'km' in df_ano.columns:
        # Converter km para numérico, tratando vírgulas
        df_ano['km'] = df_ano['km'].astype(str).str.replace(',', '.')
        df_ano['km'] = pd.to_numeric(df_ano['km'], errors='coerce')
    
    if 'br' in df_ano.columns:
        df_ano['br'] = pd.to_numeric(df_ano['br'], errors='coerce')
    
    # Colunas numéricas que podem ter problemas
    numeric_cols = ['mortos', 'feridos', 'feridos_leves', 'feridos_graves', 
                    'ilesos', 'ignorados', 'veiculos', 'pessoas']
    
    for col in numeric_cols:
        if col in df_ano.columns:
            df_ano[col] = pd.to_numeric(df_ano[col], errors='coerce').fillna(0).astype('int64')
    
    # Garantir que strings sejam strings
    string_cols = ['uf', 'municipio', 'causa_acidente', 'tipo_acidente']
    for col in string_cols:
        if col in df_ano.columns:
            df_ano[col] = df_ano[col].astype(str)
    
    return df_ano

for i, df_ano in enumerate(dataframes):
    if i < len(arquivos):
        nome_arquivo = arquivos[i].replace('.csv', '')
    else:
        nome_arquivo = arquivos_2[i - len(arquivos)].replace('.csv', '')
    
    caminho_saida = f"/Volumes/lakehouse/raw/volumes/datatran/{nome_arquivo}"
    
    # Limpar o DataFrame pandas
    df_ano_clean = clean_pandas_dataframe(df_ano.copy())
    
    # Converter para Spark DataFrame
    spark_df = spark.createDataFrame(df_ano_clean)
    
    # Salvar em Parquet
    spark_df.write.format("parquet").mode("overwrite").save(caminho_saida)
    
    print(f" Salvo: {nome_arquivo} com {len(df_ano)} registros")

 Salvo: datatran2007 com 127675 registros
 Salvo: datatran2008 com 141043 registros
 Salvo: datatran2009 com 158646 registros
 Salvo: datatran2010 com 183469 registros
 Salvo: datatran2011 com 192326 registros
 Salvo: datatran2012 com 184568 registros
 Salvo: datatran2013 com 186748 registros
 Salvo: datatran2014 com 169201 registros
 Salvo: datatran2015 com 122161 registros
 Salvo: datatran2017 com 89518 registros
 Salvo: datatran2018 com 69206 registros
 Salvo: datatran2020 com 63530 registros
 Salvo: datatran2023 com 67766 registros
 Salvo: datatran2024 com 73156 registros
 Salvo: datatran2025 com 5478 registros
